In [7]:
"""
- Constructor takes a patient_id (string)
- Implement `record(vital_type: str, value: float, timestamp: int)` 
  - Records a vital sign reading (e.g., "heart_rate", 72.0, 1700000000)
- Implement `get_latest(vital_type: str) -> float | None`
  - Returns the most recent value for that vital type, or None if no readings exist
"""

class VitalSignsMonitor:
    def __init__(self, patient_id: str):
        self.patient_id = patient_id
        self.records = {}
        self.alerts = {}

    def record(self, vital_type: str, value: float, timestamp: int) -> bool:
        if self.records.get(vital_type) is None:
            self.records[vital_type] = []
        
        self.records[vital_type].append((timestamp, value))

        if self.alerts.get(vital_type):
            min_val, max_val = self.alerts[vital_type]
            if value < min_val or value > max_val:
                return True
        
        return False

    def get_latest(self, vital_type: str) -> float | None:
        if self.records.get(vital_type) is None or not self.records[vital_type]:
            return None

        return max(self.records[vital_type], key=lambda x: x[0])[1]

    def get_average(self, vital_type: str, start_time: int, end_time:int) -> float | None:
        if self.records.get(vital_type) is None:
            return None

        # get all records matching time window:
        values = [
            value for timestamp, value in self.records[vital_type] if start_time <= timestamp <= end_time
        ]

        if not values:
            return None
        return sum(values)/len(values)

    def set_alert(self, vital_type: str, min_val: float, max_val: float) -> None:
        assert min_val <= max_val, f"min_val ({min_val}) and max_val ({max_val}) are inappropriately chosen"
        self.alerts[vital_type] = (min_val, max_val)


In [ ]:
class AppointmentScheduler:
    def __init__(self):
        self.appointments = set()
        self.uid = 0

    def overlap(self, a1, a2, b1, b2):
        if max(a1, b1) < min(a2, b2):
            return True
        return False

    def book(self, provider_id: str, patient_id: str, start_time: int, duration: int) -> str | None:
        for e_provider_id, e_patient_id, e_start_time, e_duration, e_uid in self.appointments:
            if provider_id == e_provider_id:
                # check if ranges overlap so there is a conflict
                if self.overlap(start_time, start_time + duration, e_start_time, e_start_time + e_duration):
                    return None

        self.uid += 1
        self.appointments.add((provider_id, patient_id, start_time, duration, self.uid))
        return str(self.uid)

    def get_appointment(self, appointment_id: str) -> dict | None:
        # search appointments for appointment_id
        # could switch appointments to be a dict keyed by 
        for provider_id, patient_id, start_time, duration, uid in self.appointments:
            if str(uid) == appointment_id:
                return {
                    "provider_id": provider_id, 
                    "patient_id": patient_id, 
                    "start_time": start_time, 
                    "duration": duration
                    }
        return None

    def get_available_slots(self, provider_id: str, date_start: int, date_end: int, duration: int) -> list[int]:
        taken = []
        for e_provider_id, e_patient_id, e_start_time, e_duration, e_uid in self.appointments:
            if provider_id == e_provider_id:
                # check if the times overlap: checking existing appointments between date_start and date_end
                if self.overlap(date_start, date_end, e_start_time, e_start_time + e_duration):
                    taken.append((e_start_time, e_start_time+e_duration))
        taken = sorted(taken, key=lambda x: x[0]) # sort by start time since we know existing appointments do not overlap
        
        # more pythonic:
        taken = list(
            (e_start_time, e_start_time + e_duration)
            for e_provider_id, _, e_start_time, e_duration, _ in self.appointments
            if e_provider_id == provider_id
            and self.overlap(date_start, date_end, e_start_time, e_start_time + e_duration)
        )
        taken = sorted(taken, key=lambda x: x[0]) # sort by start time since we know existing appointments do not overlap

        # brute force:
        available = []
        inc = 15
        time = ((date_start + inc-1) // inc) * inc
        while (time + duration <= date_end):
            for e_start, e_end in taken:
                if max(time, e_start) < min(time+duration, e_end):
                    break
            else:
                available.append(time)
            time += inc

        # pointer
        # since taken is monotonically increasing, we can keep track of a pointer and just check the next taken
        # instead of checking over everything. this would reduce time to worst O(m+n) best O(1) instead of O(n*m)
        available = []
        inc = 15
        time = ((date_start + inc-1) // inc) * inc
        taken_idx = 0
        while (time + duration <= date_end):
            while taken_idx < len(taken) and taken[taken_idx][1] <= time:
                taken_idx += 1
            
            # check if current slot conflicts with the next appointment
            if taken_idx < len(taken) and self.overlap(time, time+duration, taken[taken_idx][0], taken[taken_idx][1]):
                time = ((taken[taken_idx][1] + inc - 1) // inc) * inc
            else:
                available.append(time)
                time += inc

        return available


In [ ]:
"""
- Constructor takes `max_requests: int` and `window_seconds: int`
- Implement `is_allowed(client_id: str, timestamp: int) -> bool`
  - Returns True if the request should be allowed, False if rate limited
  - A client can make at most `max_requests` in any rolling `window_seconds` period
  - timestamp is in seconds (unix time)
"""
from collections import deque

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window_seconds = window_seconds

        self.requests: dict[str, deque[int]] = {} # dict of deque

    def is_allowed(self, client_id: str, timestamp: int) -> bool:
        if self.requests.get(client_id) is None:
            self.requests[client_id] = deque()

        window = self.requests[client_id] # window is deque
        if len(window) < self.max_requests: # always able to make request
            window.append(timestamp)
            return True
        else: # deque has len max_requests
            # assuming that failing here doesn't add a request, we only add on success
            time = window[0]
            if timestamp - time > self.window_seconds:
                window.append(timestamp)
                window.popleft()
                return True
        
        return False

    # we could improve the runtime of this and the above by using arrays of fixed length and a start pointer
    # and len attribute which wraps around with modulo arithmetic, but in that case we literally aren't 
    # cleaning up any space so I think this is more in keeping with the question and keeping a balance of performance
    # without unbounded window arrays
    def cleanup(self, current_time: int) -> int:
        removed = 0
        for client_id, window in self.requests.items():
            while len(window) > 0 and ((current_time - self.window_seconds) > window[0]):
                window.popleft()
                removed += 1

        return removed

In [1]:
"""
- Constructor takes `max_requests: int` and `window_seconds: int`
- Implement `is_allowed(client_id: str, timestamp: int) -> bool`
  - Returns True if the request should be allowed, False if rate limited
  - A client can make at most `max_requests` in any rolling `window_seconds` period
  - timestamp is in seconds (unix time)

- Implement `set_client_limit(client_id: str, max_requests: int, window_seconds: int)`
  - Sets a custom limit for a specific client (overrides default)
- Implement `get_usage(client_id: str, current_time: int) -> dict`
  - Returns {"requests_made": int, "requests_remaining": int, "resets_in": int}
  - resets_in = seconds until the oldest request in the window expires
  - Use client's custom limit if set, otherwise default
"""
from collections import deque

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window_seconds = window_seconds

        self.client_requests = {} # dict of deque
        self.client_max_requests = {} # dict of int
        self.client_window_seconds = {} # dict of int
    
    def set_client_limit(self, client_id: str, max_requests: int, window_seconds: int):
        self.client_max_requests[client_id] = max_requests
        self.client_window_seconds[client_id] = window_seconds 

    def is_allowed(self, client_id: str, timestamp: int) -> bool:
        if self.client_requests.get(client_id) is None:
            self.client_requests[client_id] = deque()

        if self.client_max_requests.get(client_id) is None or self.client_window_seconds.get(client_id) is None:
            self.set_client_limit(client_id, self.max_requests, self.window_seconds)

        client_window = self.client_requests[client_id] # window is deque
        client_max_requests = self.client_max_requests[client_id] # int
        client_window_seconds = self.client_window_seconds[client_id] # int

        # we might have changed the client limits; if so, remove the oldest
        while len(client_window) > client_max_requests:
            client_window.popleft()

        if len(client_window) < client_max_requests: # always able to make request
            client_window.append(timestamp)
            return True
        else: # deque has len client_max_requests
            # assuming that failing here doesn't add a request, we only add on success
            time = client_window[0]
            if timestamp - time > client_window_seconds:
                client_window.append(timestamp)
                client_window.popleft()
                return True
        
        return False

    # we could improve the runtime of this and the above by using arrays of fixed length and a start pointer
    # and len attribute which wraps around with modulo arithmetic, but in that case we literally aren't 
    # cleaning up any space so I think this is more in keeping with the question and keeping a balance of performance
    # without unbounded window arrays
    def cleanup(self, current_time: int) -> int:
        removed = 0
        for client_id, client_window in self.client_requests.items():
            client_window_seconds = self.client_window_seconds[client_id]
            while len(client_window) > 0 and ((current_time - client_window_seconds) > client_window[0]):
                client_window.popleft()
                removed += 1

        return removed
        
    # not entirely sure what to do with requests made, is it total requests made overall time? I'm going to
    # assume it's requests made in the sliding window
    def get_usage(self, client_id: str, current_time: int) -> dict:
        if self.client_requests[client_id] is None:
            return {"requests_made": 0, "requests_remaining": self.max_requests, "resets_in": self.window_seconds}
        
        client_requests = self.client_requests[client_id]
        client_max_requests = self.client_max_requests[client_id] # int
        client_window_seconds = self.client_window_seconds[client_id] # int

        requests_remaining = 0
        resets_in = 0
        for timestamp in client_requests:
            if current_time - client_window_seconds > timestamp:
                requests_remaining += 1
            else:
                resets_in = max(resets_in, client_window_seconds - (current_time - timestamp))
        
        return {"requests_made": client_max_requests-requests_remaining, "requests_remaining": requests_remaining, "resets_in": resets_in}

In [ ]:
from collections import deque

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.default_max_requests = max_requests
        self.default_window_seconds = window_seconds
        
        self.requests = {}  # client_id -> deque of timestamps
        self.limits = {}    # client_id -> (max_requests, window_seconds)
    
    def _get_limits(self, client_id: str) -> tuple[int, int]:
        if client_id in self.limits:
            return self.limits[client_id]
        return (self.default_max_requests, self.default_window_seconds)
    
    def _clean_client(self, client_id: str, current_time: int) -> int:
        if client_id not in self.requests:
            return 0
        
        _, window_seconds = self._get_limits(client_id)
        cutoff = current_time - window_seconds
        window = self.requests[client_id]
        
        removed = 0
        while window and window[0] <= cutoff:
            window.popleft()
            removed += 1
        return removed
    
    def set_client_limit(self, client_id: str, max_requests: int, window_seconds: int):
        self.limits[client_id] = (max_requests, window_seconds)
    
    def is_allowed(self, client_id: str, timestamp: int) -> bool:
        if client_id not in self.requests:
            self.requests[client_id] = deque()
        
        self._clean_client(client_id, timestamp)
        
        max_requests, _ = self._get_limits(client_id)
        window = self.requests[client_id]
        
        if len(window) < max_requests:
            window.append(timestamp)
            return True
        return False
    
    def cleanup(self, current_time: int) -> int:
        return sum(self._clean_client(cid, current_time) for cid in self.requests)
    
    def get_usage(self, client_id: str, current_time: int) -> dict:
        max_requests, window_seconds = self._get_limits(client_id)
        
        if client_id not in self.requests:
            return {"requests_made": 0, "requests_remaining": max_requests, "resets_in": 0}
        
        self._clean_client(client_id, current_time)
        window = self.requests[client_id]
        
        requests_made = len(window)
        requests_remaining = max_requests - requests_made
        resets_in = (window_seconds - (current_time - window[0])) if window else 0
        
        return {
            "requests_made": requests_made,
            "requests_remaining": requests_remaining,
            "resets_in": resets_in
        }